# Question 6

In [15]:
# To DO 
# implement the activation and its derivative

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn

# Obtain the training and validation data
training = pd.read_csv('train_data.csv').to_numpy()
x_train = training[:,0:2]
y_train = training[:,2]
print("Training Data")
print(training[0:10])

print(50*'-')

validation = pd.read_csv('validate_data.csv').to_numpy()
x_val = validation[:,0:2]
y_val = validation[:,2]
print("Validation Data")
print(validation[0:10])

def zero_mean_unit_var(x):
    x_mean = np.mean(x, axis=0)
    x_std = np.std(x, axis=0)
    x = (x-x_mean)/x_std
    return x

print (50*'-')
print (zero_mean_unit_var(x_train))

def activation(x):
    return 1/(1+np.exp(-x))


def activation_derivative(x):
    return activation(x)*(1-activation(x)) 

Training Data
[[2.91721124e+03 3.28952253e+03 0.00000000e+00]
 [1.88893772e+03 7.81528356e+02 0.00000000e+00]
 [4.18852141e+03 1.55447626e+03 0.00000000e+00]
 [8.14555534e+03 9.80406673e+03 0.00000000e+00]
 [9.58448898e+03 6.17633719e+03 0.00000000e+00]
 [4.03963376e+03 1.67607143e+02 0.00000000e+00]
 [2.39902078e+03 6.25189167e+03 1.00000000e+00]
 [5.89987684e+03 7.73251478e+03 0.00000000e+00]
 [1.83251779e+03 1.30073027e+03 0.00000000e+00]
 [1.61228897e+03 6.42678176e+03 1.00000000e+00]]
--------------------------------------------------
Validation Data
[[6.81791883e+03 7.86357780e+03 0.00000000e+00]
 [6.87936668e+03 3.90264518e+03 1.00000000e+00]
 [8.93210750e+02 6.13757242e+03 1.00000000e+00]
 [9.02868713e+03 7.44265682e+03 0.00000000e+00]
 [3.86222856e+03 7.46752602e+02 0.00000000e+00]
 [7.54177783e+03 6.94448705e+02 1.00000000e+00]
 [1.58208746e+03 7.46484261e+03 1.00000000e+00]
 [5.78893578e+02 7.18144291e+03 0.00000000e+00]
 [5.61728665e+03 4.28307909e+03 1.00000000e+00]
 [2.96

In [2]:
# To DO 
# implement the loss its derivative
def compute_loss(predicted_op, true_y):
     
    return  -true_y*np.log(predicted_op) - (1-true_y)*np.log(1-predicted_op)

def loss_derivative(x):
    return 

In [3]:
# TO DO 
# organize the training data into batches
def make_batches(x_train, y_train, batch_size):

    return x_batches, y_batches

In [4]:
# TO DO 
def forward_pass(x_batch, y_batch):

  return predicted_op


In [5]:
# TO DO 
def backward_pass(loss):

  return grads

In [7]:
#TO DO
def update_weights(weights, grads, lr):

  return

In [8]:
# To Do
# define you stopping criteria for training
def stopping_criteria():
  stop = False
  return stop

In [9]:
# TO DO
# Define the hyperparameters. Add any additional hyperparameters you might need
lr = 0  # learning rate
batch_size = 0
num_epochs = 0

In [10]:
def train(x_train, y_train, lr, batch_size, num_epochs):

  while not stopping_criteria():
      # TO DO 
      # get a batch from the data
      x_batch, y_batch = [], []

      # forward the batch through the network layers
      predicted_op = forward_pass(x_batch)

      # compute the loss
      loss = compute_loss(predicted_op, y_batch)

      # perform backward pass
      grads = backward_pass(loss)

      # update the weights of the network
      update_weights(network, grads, lr)
